### Исследование данных об инвестиции венчурных фондов в компании-стартапы

1) Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

In [ ]:
SELECT funding_total
FROM company
WHERE category_code LIKE ('news') AND country_code LIKE ('USA')
ORDER BY funding_total DESC

2) Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash' 
     AND EXTRACT (YEAR FROM CAST(acquired_at AS timestamp)) BETWEEN '2011' AND '2013'

3) Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE ('Silver%')

4) Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code,
       SUM(funding_total)
FROM company 
GROUP BY country_code
ORDER BY SUM(funding_total) DESC

5) Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at,
        MIN(raised_amount) AS min,
        MAX(raised_amount) AS max
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) <> 0 AND (MIN(raised_amount) <> MAX(raised_amount))

6) Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,
        CASE 
          WHEN invested_companies >= 100 THEN 'high_activity'
          WHEN invested_companies < 100 AND invested_companies >= 20 THEN 'middle_activity'
          WHEN invested_companies < 20 THEN 'low_activity'
        END
FROM fund

7) Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds)) AS avg
FROM fund
GROUP BY activity
ORDER BY avg 

8) Выгрузите таблицу с десятью самыми активными инвестирующими странами. Активность страны определите по среднему количеству компаний, в которые инвестируют фонды этой страны.
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды, основанные с 2010 по 2012 год включительно.
Исключите из таблицы страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Отсортируйте таблицу по среднему количеству компаний от большего к меньшему.

In [ ]:
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE EXTRACT (YEAR FROM CAST(founded_at AS date)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) <> 0
ORDER BY AVG(invested_companies) DESC
LIMIT 10

9) Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT OUTER JOIN education AS e ON p.id = e.person_id

10) Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
SELECT DISTINCT c.name,
       COUNT(DISTINCT e.instituition) AS inst
FROM company AS c
INNER JOIN people AS p ON c.id = p.company_id
INNER JOIN education AS e ON p.id = e.person_id
GROUP BY c.name
ORDER BY inst DESC
LIMIT 5

11) Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT c.name
FROM company AS c
JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.status = 'closed' AND fr.is_last_round = 1 AND fr.is_first_round = 1

12) Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT DISTINCT p.id
FROM people AS p
WHERE p.company_id IN (SELECT DISTINCT c.id
                      FROM company AS c
                      JOIN funding_round AS fr ON c.id = fr.company_id
                      WHERE c.status = 'closed' AND fr.is_last_round = 1 AND fr.is_first_round = 1)



13) Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT DISTINCT p.id,e.instituition
FROM people AS p
JOIN education AS e ON p.id = e.person_id
WHERE p.company_id IN (SELECT DISTINCT c.id
                      FROM company AS c
                      JOIN funding_round AS fr ON c.id = fr.company_id
                      WHERE c.status = 'closed' AND fr.is_last_round = 1 AND fr.is_first_round = 1)

14) Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания.

In [ ]:
SELECT DISTINCT p.id,
       COUNT(e.instituition)
FROM people AS p
JOIN education AS e ON p.id = e.person_id
WHERE p.company_id IN (SELECT DISTINCT c.id
                      FROM company AS c
                      JOIN funding_round AS fr ON c.id = fr.company_id
                      WHERE c.status = 'closed' AND fr.is_last_round = 1 AND fr.is_first_round = 1)
GROUP BY p.id

15) Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
SELECT AVG(w.count_inst)
FROM(SELECT COUNT(e.instituition) AS count_inst
    FROM people AS p
    JOIN education AS e ON p.id = e.person_id
    WHERE p.company_id IN (SELECT DISTINCT c.id
                          FROM company AS c
                          JOIN funding_round AS fr ON c.id = fr.company_id
                          WHERE c.status = 'closed' AND fr.is_last_round = 1 AND fr.is_first_round = 1)
    GROUP BY p.id) AS w

16) Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники компании Facebook.

In [ ]:
SELECT AVG(w.count_inst)
FROM(SELECT COUNT(e.instituition) AS count_inst
    FROM people AS p
    JOIN education AS e ON p.id = e.person_id
    WHERE p.company_id IN (SELECT c.id
                          FROM company AS c
                          WHERE c.name = 'Facebook')
    GROUP BY p.id) AS w

17) Составьте таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM investment AS inv
LEFT OUTER JOIN company AS c ON inv.company_id = c.id
LEFT OUTER JOIN fund AS f ON inv.fund_id = f.id
LEFT OUTER JOIN funding_round AS fr ON inv.funding_round_id = fr.id
WHERE c.milestones > 6 AND EXTRACT(YEAR FROM fr.funded_at) IN (2012,2013)

18) Выгрузите таблицу, в которой будут такие поля:
название компании-покупателя;
сумма сделки;
название компании, которую купили;
сумма инвестиций, вложенных в купленную компанию;
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в алфавитном порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT c.name,
       ac.price_amount,
       co.name,
       co.funding_total,
       ROUND(ac.price_amount/co.funding_total) AS perc
FROM acquisition AS ac
LEFT OUTER JOIN company AS c ON ac.acquiring_company_id  = c.id 
LEFT OUTER JOIN company AS co ON ac.acquired_company_id = co.id
WHERE co.funding_total <> 0 AND ac.price_amount <> 0
ORDER BY ac.price_amount DESC, co.name
LIMIT 10

19) Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name,
       EXTRACT(MONTH FROM fr.funded_at)
FROM company AS c
LEFT OUTER JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.category_code = 'social' AND 
      EXTRACT(YEAR FROM fr.funded_at) BETWEEN 2010 AND 2013

20) Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
количество уникальных названий фондов из США, которые инвестировали в этом месяце;
количество компаний, купленных за этот месяц;
общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH
a AS (SELECT EXTRACT(MONTH FROM acquired_at) AS months,
              COUNT(acquired_company_id) AS b_comp,
               SUM(price_amount) AS sum_a
        FROM acquisition AS a
        WHERE EXTRACT(YEAR FROM acquired_at) BETWEEN 2010 AND 2013 
        GROUP BY EXTRACT(MONTH FROM acquired_at)),
        
f AS (SELECT EXTRACT (MONTH FROM fr.funded_at) AS months,
       COUNT(DISTINCT f.name) AS dist_funds       
FROM funding_round AS fr
LEFT OUTER JOIN investment AS i ON fr.id = i.funding_round_id
LEFT OUTER JOIN fund AS f ON i.fund_id = f.id
WHERE f.country_code = 'USA' AND EXTRACT (YEAR FROM fr.funded_at) BETWEEN 2010 AND 2013
GROUP BY months)


SELECT f.months,
       f.dist_funds,
       a.b_comp,
       a.sum_a
FROM f
JOIN a ON f.months = a.months

21) Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
c_11 AS (SELECT country_code AS country,
               AVG(funding_total) AS avg_11
        FROM company
        WHERE EXTRACT(YEAR FROM founded_at) = 2011
        GROUP BY country_code),
c_12 AS (SELECT country_code AS country,
               AVG(funding_total) AS avg_12
        FROM company
        WHERE EXTRACT(YEAR FROM founded_at) = 2012
        GROUP BY country_code),
c_13 AS (SELECT country_code AS country,
               AVG(funding_total) AS avg_13
        FROM company
        WHERE EXTRACT(YEAR FROM founded_at) = 2013
        GROUP BY country_code)
        
SELECT c_11.country,
       c_11.avg_11,
       c_12.avg_12,
       c_13.avg_13
FROM c_11
JOIN c_12 ON c_11.country = c_12.country
JOIN c_13 ON c_12.country = c_13.country
ORDER BY c_11.avg_11 DESC